In [ ]:
# wordcloud and boost specific words?
# statsmodels, ordinary least squares, pvalue



In [1]:
import pandas as pd
import numpy as np
from string import punctuation
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE


In [2]:
fname = "../data/S1_8Sharktankpitchesdeals.csv"
data = pd.read_csv(fname)
data

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Valuation,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,...,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,333333.0,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,...,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
1,101,College Foxes Packing Boxes,250000,0.25,1000000.0,Male,Lifestyle / Home,a packing and organizing service based on an a...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,101,Ionic Ear,1000000,0.15,6666667.0,Male,Software / Tech,an implantable Bluetooth device requiring surg...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,4600000.0,Male,Food and Beverage,a pie company,1,0,...,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
4,101,Wispots,1200000,0.10,12000000.0,Male,Business Services,an electronic hand-held device for waiting roo...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,824,Wallet Buckle,500000,0.10,5000000.0,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,...,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
700,826,Bridal Buddy,75000,0.10,750000.0,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,...,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN
701,826,Laid Brand,30000,0.30,100000.0,Male,Fashion / Beauty,hair-care products made with pheromones . Laid...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
702,826,Rocketbook,400000,0.10,4000000.0,Male,Software / Tech,a notebook that can scan contents to cloud ser...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [3]:
# Choose model output

# "Deal_Status"
# "Deal_Shark 1"

# "Kevin O'Leary"
# "Barbara Corcoran"
# "Robert Herjavec"
# "Daymond John"
# "Lori Greiner"
# "Mark Cuban"
# "Others"
shark_to_model = "Others"

In [4]:
# drop rows where shark_to_model is Nan
data=data.dropna(subset=[shark_to_model])
# .reset_index(drop=True)
data

,Season_Epi_code,Pitched_Business_Identifier,Amount_Asked_For,Exchange_For_Stake,Valuation,Gender,Category,Pitched_Business_Desc,Deal_Status,Kevin O'Leary,...,Mark Cuban,Robert Herjavec,Daymond John,Barbara Corcoran,Others,Deal_Shark 1,Deal_Shark 2,Deal_Shark 3,Deal_Shark 4,Deal_Shark 5
0,101,Ava the Elephant,50000,0.15,333333.0,Female,Health / Wellness,"(Emmy the Elephant during show, trademarked a...",1,0,...,0,0,0,1,0,Barbara Corcoran,NaN,NaN,NaN,NaN
1,101,College Foxes Packing Boxes,250000,0.25,1000000.0,Male,Lifestyle / Home,a packing and organizing service based on an a...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
2,101,Ionic Ear,1000000,0.15,6666667.0,Male,Software / Tech,an implantable Bluetooth device requiring surg...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,101,Mr. Tod's Pie Factory,460000,0.10,4600000.0,Male,Food and Beverage,a pie company,1,0,...,0,0,1,1,0,Barbara Corcoran,Daymond John,NaN,NaN,NaN
4,101,Wispots,1200000,0.10,12000000.0,Male,Business Services,an electronic hand-held device for waiting roo...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,824,Wallet Buckle,500000,0.10,5000000.0,Male,Fashion / Beauty,a belt buckle that holds credit cards . The Wa...,1,0,...,0,1,0,0,0,Robert Herjavec,NaN,NaN,NaN,NaN
700,826,Bridal Buddy,75000,0.10,750000.0,Female,Fashion / Beauty,a functional slip worn under a wedding gown th...,1,1,...,0,0,0,0,0,Kevin O'Leary,Lori Greiner,NaN,NaN,NaN
701,826,Laid Brand,30000,0.30,100000.0,Male,Fashion / Beauty,hair-care products made with pheromones . Laid...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
702,826,Rocketbook,400000,0.10,4000000.0,Male,Software / Tech,a notebook that can scan contents to cloud ser...,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN


In [5]:
# Custom tokenizer from class activity 22-1-4
stopwords = stopwords.words( 'english' ) + list(punctuation)
stemmer = PorterStemmer()
# Stemming
punc_list = list(punctuation)
def special_remove(word):
    if len(word)>2:
        return False
    for c in word:
        if c in punc_list:
            return True
    return False

# custom function that overrides default token generation
def custom_tokenizer(text):
    text = text.lower()
    words = nltk.word_tokenize(text)
    words = [stemmer.stem(w) for w in words if w not in stopwords+["..."]]
    # further remove words with a special char
    words = [w for w in words if not special_remove(w)]
    return words

activity_vectorizer = TfidfVectorizer(ngram_range=(1,2)
                            ,tokenizer=custom_tokenizer 
                            ,stop_words='english'
                            # ,min_df=2
                            ,strip_accents='unicode'
                            )

activity_vectors = activity_vectorizer.fit_transform(data["Pitched_Business_Desc"])
activity_df = pd.DataFrame(activity_vectors.toarray(), columns=activity_vectorizer.get_feature_names())
activity_df

C:\Users\atk285\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'thu', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,-not,-not simpli,"1,2pee",10,10 cool,10 day,10 minut,10 net,10 spandex,"10,000",...,“ total,“ uv,“ wrap,“ year,“ yuk,”,” day,” peopl,” potato,” prevent
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
activity_vectorizer.get_feature_names()

['-not',
 '-not simpli',
 '1,2pee',
 '10',
 '10 cool',
 '10 day',
 '10 minut',
 '10 net',
 '10 spandex',
 '10,000',
 '10,000 run',
 '10-30',
 '10-30 minut',
 '100',
 '100 built-in',
 '100 calori',
 '100 food',
 '100 guarante',
 '100 mile',
 '100 natur',
 '100 recycl',
 '100 time',
 '100 zinc',
 '100-calori',
 '100-calori pint',
 '11',
 '11 year',
 '12',
 '12 fashion',
 '12 pack',
 '123',
 '123 plush',
 '15',
 '15 flavor',
 '15 throw',
 '15 year',
 '15/30',
 '15/30 spf',
 '150',
 '150 90',
 '2',
 '2 ipad',
 '2 type',
 '2 week',
 '2-5',
 '2-5 age',
 '2-in-1',
 '2-in-1 spray',
 '20',
 '20 week',
 '20-30',
 '20-30 second',
 '2009',
 '2009 11',
 '2012',
 '2012 beardbrand',
 '2012 pipcorn',
 '20mph',
 '20mph 30mph',
 '21st',
 '21st centuri',
 '23',
 '23 ounc',
 '24',
 '24 mph',
 '2400',
 '2400 expert',
 '2400 sat',
 '24—24',
 '24—24 board',
 '26',
 '26 pound',
 '26g',
 '26g protein',
 '270',
 '270 shoe',
 '2700',
 '2700 diaper',
 '3',
 '3 design',
 '3 ipad',
 '3 pack',
 '3-d',
 '3-d design',

In [7]:
# Choose features

# To include Pitch, comment in next line, and comment out the following line
# To exclude Pitch, comment out next line, and comment in the following line

selected_features = activity_df
# selected_features = pd.DataFrame([])

selected_features["Gender"]=data["Gender"]
selected_features["Category"]=data["Category"]
selected_features["Amount_Asked_For"]=data["Amount_Asked_For"]
selected_features["Exchange_For_Stake"]=data["Exchange_For_Stake"]
selected_features["Valuation"]=data["Valuation"]
selected_features

,-not,-not simpli,"1,2pee",10,10 cool,10 day,10 minut,10 net,10 spandex,"10,000",...,”,” day,” peopl,” potato,” prevent,Gender,Category,Amount_Asked_For,Exchange_For_Stake,Valuation
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Female,Health / Wellness,50000,0.15,333333.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Lifestyle / Home,250000,0.25,1000000.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Software / Tech,1000000,0.15,6666667.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Food and Beverage,460000,0.10,4600000.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Business Services,1200000,0.10,12000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,500000,0.10,5000000.0
700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Female,Fashion / Beauty,75000,0.10,750000.0
701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Fashion / Beauty,30000,0.30,100000.0
702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Male,Software / Tech,400000,0.10,4000000.0


In [8]:
selected_features = pd.get_dummies(selected_features)
selected_features

,-not,-not simpli,"1,2pee",10,10 cool,10 day,10 minut,10 net,10 spandex,"10,000",...,Category_Fitness / Sports / Outdoor,Category_Food and Beverage,Category_Green / Clean Tech,Category_Health / Wellness,Category_Lifestyle / Home,Category_Media / Entertainment,Category_Pet Products,Category_Software / Tech,Category_Travel,Category_Uncertain / Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
701,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
selected_features.columns

Index(['-not', '-not simpli', '1,2pee', '10', '10 cool', '10 day', '10 minut',
       '10 net', '10 spandex', '10,000',
       ...
       'Category_Fitness / Sports / Outdoor', 'Category_Food and Beverage',
       'Category_Green / Clean Tech', 'Category_Health / Wellness',
       'Category_Lifestyle / Home', 'Category_Media / Entertainment',
       'Category_Pet Products', 'Category_Software / Tech', 'Category_Travel',
       'Category_Uncertain / Other'],
      dtype='object', length=13190)

In [10]:
model = MultinomialNB()
X_smote, y_smote = SMOTE().fit_sample(selected_features, data[[shark_to_model]].values.ravel())
X = X_smote
y = y_smote

# X = selected_features
# y = data[[shark_to_model]].values.ravel()
print(X.shape)
print(y.shape)

(1336, 13190)
(1336,)


In [11]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2, f_classif
# #apply SelectKBest class to extract top 10 best features
# bestfeatures = SelectKBest(score_func=f_classif, k=6)
# fit = bestfeatures.fit(X,y)
# dfscores = pd.DataFrame(fit.scores_)
# dfpvalues = pd.DataFrame(fit.pvalues_)
# dfcolumns = pd.DataFrame(X.columns)
# #concat two dataframes for better visualization 
# featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
# featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
# print(featureScores.nlargest(20,'Score'))  #print 10 best features

In [12]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2, f_classif
# #apply SelectKBest class to extract top 10 best features
# bestfeatures = SelectKBest(score_func=chi2, k=6)
# fit = bestfeatures.fit(X,y)
# dfscores = pd.DataFrame(fit.scores_)
# dfpvalues = pd.DataFrame(fit.pvalues_)
# dfcolumns = pd.DataFrame(X.columns)
# #concat two dataframes for better visualization 
# featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
# featureScores.columns = ['Specs','Score','PValue']  #naming the dataframe columns
# print(featureScores.nlargest(20,'Score'))  #print 10 best features

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [14]:
model.fit(X_train, y_train.ravel())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
y_pred = model.predict(X_test)

In [16]:
pd.DataFrame({"actual": y_test.reshape(-1), "prediction": y_pred.reshape(-1)})

,actual,prediction
0,1,1
1,1,0
2,1,1
3,0,0
4,0,0
...,...,...
329,0,1
330,1,0
331,0,0
332,1,0


In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5568862275449101

In [18]:
# Import ML models from sklearn
from sklearn.linear_model import LogisticRegression # Regression classifier
from sklearn.tree import DecisionTreeClassifier # Decision Tree classifier
from sklearn import svm # Support Vector Machine
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent Classifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # Random Forest and Gradient Boosting Classifier
from sklearn.naive_bayes import MultinomialNB # Naive Bayes Classifier 
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix # Some metrics to check the performance of the models

In [19]:
# Setting parameters for each algorithm - these are tunable to achieve max accuracy

Classifiers = {'LR':LogisticRegression(random_state=10,C=5,max_iter=300, solver='lbfgs')
               ,'DTC':DecisionTreeClassifier(random_state=10,min_samples_leaf=2)
               ,'RF':RandomForestClassifier(random_state=10,n_estimators=100,n_jobs=-1)
               # ,'GBC':GradientBoostingClassifier(random_state=10,n_estimators=400,learning_rate=0.2)
               ,'SGD':SGDClassifier(loss="hinge", penalty="l2")
               # ,'SVM':svm.SVC(kernel='linear', C=0.1)
               ,'NB':MultinomialNB(alpha=.05)
               }

In [20]:
Classifiers.keys()

dict_keys(['LR', 'DTC', 'RF', 'SGD', 'NB'])

In [21]:
# Create a pipeline so you can reuse the code

def ML_Pipeline(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    Accuracy = accuracy_score(y_test,y_pred)
    Precision = metrics.precision_score(y_test,y_pred)
    Recall = recall_score(y_test,y_pred)
    Confusion_matrix = confusion_matrix(y_test,y_pred)
    print('Classifier = '+(clf_name))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20) 
    return clf

In [22]:
clf_dict = []
for clf_name in Classifiers.keys():
    clf = Classifiers[clf_name]
    fit = clf.fit(X_train, y_train.ravel())
    y_pred = clf.predict(X_test)
    Accuracy = accuracy_score(y_test,y_pred)
    Precision = metrics.precision_score(y_test,y_pred)
    Recall = recall_score(y_test,y_pred)
    Confusion_matrix = confusion_matrix(y_test,y_pred)
    clf_dict.append({'Classifier': clf_name, 'Accuracy': Accuracy, 'Precision': Precision, 'Recall': Recall})
    print('Classifier = '+(clf_name))
    print('Accuracy = '+str(Accuracy))
    print('Precision = '+str(Precision))
    print('Recall = '+str(Recall))
    print(Confusion_matrix)
    print('==='*20)


Classifier = LR
Accuracy = 0.5269461077844312
Precision = 0.7777777777777778
Recall = 0.04294478527607362
[[169   2]
 [156   7]]
Classifier = DTC
Accuracy = 0.9520958083832335
Precision = 0.9454545454545454
Recall = 0.9570552147239264
[[162   9]
 [  7 156]]
Classifier = RF
Accuracy = 0.9940119760479041
Precision = 1.0
Recall = 0.9877300613496932
[[171   0]
 [  2 161]]
Classifier = SGD
Accuracy = 0.5119760479041916
Precision = 0.0
Recall = 0.0
[[171   0]
 [163   0]]
Classifier = NB
Accuracy = 0.5568862275449101
Precision = 0.6153846153846154
Recall = 0.24539877300613497
[[146  25]
 [123  40]]
C:\Users\atk285\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
clf_df = pd.DataFrame(clf_dict)
clf_df

,Classifier,Accuracy,Precision,Recall
0,LR,0.526946,0.777778,0.042945
1,DTC,0.952096,0.945455,0.957055
2,RF,0.994012,1.000000,0.987730
3,SGD,0.511976,0.000000,0.000000
4,NB,0.556886,0.615385,0.245399


In [24]:
clf_df['Accuracy'].max()

0.9940119760479041

In [25]:
best_clf = clf_df.loc[clf_df['Accuracy'].idxmax(),'Classifier']
best_clf

'RF'

In [26]:
best_model = ML_Pipeline(best_clf)
best_model


Classifier = RF
Accuracy = 0.9940119760479041
Precision = 1.0
Recall = 0.9877300613496932
[[171   0]
 [  2 161]]


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [27]:
from sklearn.externals import joblib
joblib.dump(best_model, f"{shark_to_model}_model.pkl")

C:\Users\atk285\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['Others_model.pkl']

In [28]:
import pickle
pickle.dump(selected_features.columns,open(f"{shark_to_model}_vocab.pkl","wb"))